In [5]:
# Imports

import nltk
import requests
import time
import pandas as pd
import os
import re
import gensim
import gensim.corpora as corpora
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import streamlit as st
import torch
import numpy as np
import tensorflow as tf
import gensim.downloader as api
import tensorflow_hub as hub
import nlpaug.augmenter.word as naw
import random
import warnings
import torch.nn.functional as F

from torch.utils.tensorboard import SummaryWriter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from textblob import TextBlob
from collections import Counter
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoTokenizer
from langchain import PromptTemplate, LLMChain
from dotenv import find_dotenv, load_dotenv
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, Word2Vec, KeyedVectors
from sklearn.manifold import TSNE
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorboard.plugins import projector
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence
from tensorboard.plugins import projector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model, Sequential
from scipy.spatial.distance import euclidean, cosine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from nlpaug.util import Action
from scipy import spatial

load_dotenv()

# ---------- Functions ---------- #

# Preprocessing function
def preprocessing(text):
    # Corrected spelling on lower case text
    corrected_text = str(TextBlob(text.lower()).correct())

    return corrected_text

# ---------- Loading the dataset ---------- #

df = pd.read_csv('/Users/alexandrecogordan/Documents/ESILV/Ongoing/Machine Learning For NLP/Project 2/NLP_Data_Exploration_and_Modeling/yelp_reviews.csv')

df.drop_duplicates(inplace=True)
df.dropna(subset=['text', 'rating', 'location'], inplace=True)

# ---------- Preprocessing ---------- #

stop_words = set(stopwords.words('english'))

def review_to_vector(review, model):
    words = review.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

KeyError: 0

In [13]:
def semantic_search(query, model, reviews, topn=10):
    query_vector = review_to_vector(query, model)

    # Calculate similarity between query and each review
    similarities = []

    restaurant_set = set()

    for review in reviews:
        review_vector = review_to_vector(review, model)
        similarity = 1 - spatial.distance.cosine(query_vector, review_vector)
    
        # ## 

        # Find the row in the df dataframe that corresponds to the review
        row = df[df['cleaned_text'] == review]
        restaurant_id = row['restaurant_id'].values[0]
        
        if restaurant_id not in restaurant_set:
            similarities.append((review, similarity))
            restaurant_set.add(restaurant_id)

        # ##

        # Replace with this if this doesn't work
            
        # similarities.append((review, similarity))

    # Sort reviews by similarity
    sorted_reviews = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Return the topn most similar reviews
    return sorted_reviews[:topn]

model = api.load('glove-twitter-50') # Change to 200 when we have the time

user_query = "Food for families and friends"
search_results = semantic_search(user_query, model, df['cleaned_text'].tolist())

In [25]:
df['text'][2]

"Been here a few times, in just recent weeks. This doesn't count visits to the rooftop bar so it's time. Time to drop my thoughts of my experiences. Here..."